## ファイル操作

* カレントディレクトリをホームに変更

In [1]:
cd(homedir())

## ファイル/フォルダ/シンボリックリンクの作成

* Emptyという空フォルダを作成

In [2]:
mkdir("Empty")

SystemError: SystemError (with Empty): mkdir: File exists

* 一時ディレクトリの場合
```Julia
mktempdir() # ディレクトリのパスを返す
```

* Blankという空ファイルを作成

In [3]:
touch("Blank")

"Blank"

* フォルダEmptyの中にファイルBlankのシンボリックリンクSymlinkを作成

In [4]:
symlink("Blank","Empty/Symlink")

Base.IOError: IOError: symlink: file already exists (EEXIST)

## 書込み
* Untitled.mdというMarkdownファイルを作成して書込み

In [5]:
io=open("Untitled.md","w")
write(io,"# Header 1")
close(io)

## 移動/名称変更

* フォルダEmptyをPackageに名称変更

In [6]:
mv("Empty","Package")

"Package"

* Packageフォルダ内のSymlinkファイルをAliasに名称変更

In [7]:
mv("Package/Symlink","Package/Alias")

"Package/Alias"

* Untitled.mdを移動して,名称変更

In [8]:
mv("Untitled.md","Package/Headers.md")

"Package/Headers.md"

## 追記
* Markdownファイルに追記

In [9]:
io=open("Package/Headers.md","a")
write(io,"\n## Header 2\n### Header 3")
close(io)

## 読込み
* Markdownファイルを読込み

In [10]:
io=open("Package/Headers.md","r")
print(read(io,String))
close(io)

# Header 1
## Header 2
### Header 3

## 再帰的にフォルダ作成
* フォルダを一気に作成

In [11]:
mkpath("Package/Module/Submodule/Item")

"Package/Module/Submodule/Item"

- `mkdir` の代わりに `mkpath` を使うことで,作成するフォルダItemの上位フォルダModule,Submoduleが存在していなくても,同時に生成される

## ディレクトリの内容を表示

In [12]:
readdir("Package")

3-element Array{String,1}:
 "Alias"     
 "Headers.md"
 "Module"    

## ファイル/フォルダの複製
* BlankとModuleを複製

In [13]:
cp("Blank","Package/Blank")
cp("Package/Module","Package/Module Copy")

"Package/Module Copy"

## ファイル/フォルダの削除

* ファイルBlankを削除

In [14]:
rm("Blank")

* シンボリックリンクAliasを削除

In [15]:
rm("Package/Alias")

* フォルダModuleを削除

In [16]:
rm("Package/Module",recursive=true)

- 空のフォルダであればそのまま削除できる。空でない場合は削除できない。
	`recursive=true` を追加すれば,空でなくても削除できる
- `force=true` を追加すれば,ファイル/フォルダが実在しない場所を指定してもエラーを返さない

## 権限を確認/変更

* 状態を確認する関数

In [17]:
function check(path::String)
	if islink(path)
		println("シンボリックリンクです")
		println("リンク先: $(readlink(path))")
		if isfile(path)
			println("リンク先はファイルです")
		end
		if isdir(path)
			print("リンク先はフォルダです")
		end
	else
		if isfile(path)
			println("ファイルです")
		end
		if isdir(path)
			println("フォルダです")
		end
	end
	if uperm(path)&04>0
		println("読込可能です")
	end
	if uperm(path)&02>0
		println("書込可能です")
	end
	if uperm(path)&01>0
		println("実行可能です")
	end
	if filesize(path)==0
		println("空です")
	end
end

check (generic function with 1 method)

* Markdownファイルの現在の状態を確認

In [18]:
check("Package/Headers.md")

ファイルです
読込可能です
書込可能です


* 読込/書込権限を剥奪

In [19]:
chmod("Package/Headers.md",0o044)

"Package/Headers.md"

* 状態を確認

In [20]:
check("Package/Headers.md")

ファイルです


* 読込/実行権限を付加

In [21]:
chmod("Package/Headers.md",0o544)

"Package/Headers.md"

* 状態を確認

In [22]:
check("Package/Headers.md")

ファイルです
読込可能です
実行可能です


- `chmod` は数値により権限を指定する
	* 4: 読込可能 (Readable)
	* 2: 書込可能 (Writable)
	* 1: 実行可能 (eXecutable)

- これらの和を3つ並べて指定する。  
    1つ目はユーザの権限,2つ目はゲストの権限,3つ目はその他の者の権限である。  
	Juliaでは8進数で表記するために,前に `0o` を付加する

	e.g. `chmod(somefile,0o754)`
		* ユーザは 7 = 4+2+1 だから,読込可能,書込可能,実行可能
		* ゲストは 5 = 4 + 1 だから,読込可能,実行可能
		* その他は 4 だから,読込のみ可能